In [1]:
import pandas as pd
from tqdm import tqdm_notebook
import pyproj
from functools import partial
from shapely.geometry import Polygon, Point
from shapely.ops import transform
from shapely.wkt import loads
import plotly.express as px

In [2]:
df_filter_setores = pd.read_csv('df_filter_setores.csv')

FileNotFoundError: File b'df_filter_setores.csv' does not exist

In [ ]:
df_filter_setores.shape

In [ ]:
df_filter_setores.columns

In [ ]:
df_filter_setores.head()

In [ ]:
fig = px.histogram(df_filter_setores, x="renda_nominal", marginal="box")
fig.show()

In [ ]:
fig = px.histogram(df_filter_setores, x="domicilios", marginal="box")
fig.show()

In [ ]:
def geodesic_point_buffer(lat, lon, meters):
    proj_wgs84 = pyproj.Proj(init='epsg:4326')
    aeqd_proj = '+proj=aeqd +lat_0={lat} +lon_0={lon} +x_0=0 +y_0=0'
    project = partial(pyproj.transform, pyproj.Proj(aeqd_proj.format(
        lat=lat, lon=lon)), proj_wgs84)
    buf = Point(0, 0).buffer(meters)
    return Polygon(transform(project, buf).exterior.coords[:])

In [ ]:
isocota1 = geodesic_point_buffer(-23.56184276, -46.68466877, 500)
isocota1

In [ ]:
geoms = []
for i, data in tqdm_notebook(df_filter_setores.iterrows(), total=df_filter_setores.shape[0]):
    geom_wkt = loads(data['geom'])
    if geom_wkt.intersects(isocota1):
        geoms.append(data)
df_instersects = pd.DataFrame(geoms)

In [ ]:
df_instersects.reset_index(drop=True, inplace=True)
df_instersects.head()

In [ ]:
isocota1.intersection(loads(df_instersects.geom[2]))

In [ ]:
isocota1.intersection(loads(df_instersects.geom[2])).area/loads(df_instersects.geom[2]).area*100

In [ ]:
for i, data in tqdm_notebook(df_instersects.iterrows(), total=df_instersects.shape[0]):
    rendas_sum = 0
    domicilios_sum = 0
    proportion = isocota1.intersection(loads(df_instersects.geom[i])).area/loads(df_instersects.geom[i]).area
    rendas_sum += proportion*df_instersects['renda_nominal'].loc[i]
    domicilios_sum += proportion*df_instersects['domicilios'].loc[i]
    buffer = pd.Series()
    buffer['geom'] = isocota1.wkt
    buffer['renda_nominal'] = rendas_sum
    buffer['domicilios'] = domicilios_sum
buffer['renda_media'] = buffer['renda_nominal']/buffer['domicilios']

In [ ]:
buffer

In [ ]:
coords = {0: {'lat': -23.57883506, 'lng': -46.68939816}, 
          1: {'lat': -23.55447951, 'lng': -46.64217924}, 
          2: {'lat': -23.53939352, 'lng': -46.69291126}}

In [ ]:
df_buffers = pd.DataFrame.from_dict(coords, orient='index')

In [ ]:
for i_buffer, coord in tqdm_notebook(df_buffers.iterrows(), total=df_buffers.shape[0]):
    rendas = np.zeros((df_filter_setores.shape[0],))
    domicilios = np.zeros((df_filter_setores.shape[0],))
    lat = coord['lat']
    lng = coord['lng']
    isocota = geodesic_point_buffer(lat, lng, 500)
    df_buffers.loc[i_buffer, 'geom'] = isocota.wkt
    for i, data in tqdm_notebook(df_filter_setores.iterrows(), total=df_filter_setores.shape[0]):
        if loads(data.geom).intersects(isocota):
            proportion = loads(data.geom).intersection(isocota).area/loads(data.geom).area
            rendas[i] = proportion*df_filter_setores['renda_nominal'].loc[i]
            domicilios[i] = proportion*df_filter_setores['domicilios'].loc[i]
    df_buffers.loc[i_buffer, 'renda_nominal'] = rendas.sum()
    df_buffers.loc[i_buffer, 'domicilios'] = domicilios.sum()
df_buffers['renda_media'] = df_buffers['renda_nominal']/df_buffers['domicilios']

In [ ]:
df_buffers

In [ ]:
df_buffers['loc'] = df_buffers.apply(lambda x: ', '.join([str(x['lat']), str(x['lng'])]), axis=1)

In [ ]:
df_buffers

In [ ]:
fig = px.scatter_3d(df_buffers, x="renda_nominal", 
                    y="domicilios", 
                    z="renda_media", 
                    hover_name="loc")
fig.show()